In [14]:
%pip install -q llama-index-llms-openai
%pip install -q llama-index-embeddings-openai
%pip install -q llama-index-finetuning
%pip install -q llama-index-readers-file
%pip install -q llama-index-llms-huggingface
%pip install -q llama-index-embeddings-huggingface
%pip install -q llama-index-vector-stores-faiss
%pip install -q chromadb
%pip install -q llama_index_vector_stores.chroma

In [ ]:
!mkdir ./splitted

In [ ]:
import pandas as pd

#data splited for easier retiriever evaluation

f = pd.read_csv("./medium.csv")

pth = './splitted/'
for n,i in enumerate(f.values):
    ff = open(pth+str(n)+'.txt','w')
    ff.write(i[0]+'\n')
    ff.write(i[1])

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from dotenv import load_dotenv
import numpy as np
import os

load_dotenv()
#open documents
documents = SimpleDirectoryReader("./splitted").load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

#spliting text for smaller contexts
text_splitter = SentenceSplitter(chunk_size=200, chunk_overlap=10)

#creating nodes of contexts with relations with similar context
nodes = text_splitter.get_nodes_from_documents(documents=documents)

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import StorageContext

#creating database fo vector storage

chroma_client = chromadb.PersistentClient('./medium_chroma1')
chroma_collection = chroma_client.create_collection("test1")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#loads BAAI/bge-small-en-v1.5
#for creating embaddings
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#creating indexing algorithm
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, embed_model=embed_model)

In [ ]:
#making retriever for data structure of nodes
retriever = index.as_retriever(similarity_top_k=5)

In [ ]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

#model for chatting
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)

In [ ]:
query_engine = index.as_query_engine(llm=llm)

In [ ]:
from llama_index.core import PromptTemplate

#creating templates with roles for chat

template = """Article: {question}

    Answer: You are an AI expert who prepares questions for a test about AI. Questions:"""

new_summary_tmpl = PromptTemplate(template)



new_summary_tmpl_str = (
    "You always say 'Hello my friend' at the beginning of your answer. Below you find data from a database\n"
    "{context_str}\n"
    "Take that context and try to answer the question with it."
    "Query: {query_str}\n"
    "Answer: "
)
new_summary_tmpl = PromptTemplate(new_summary_tmpl_str)


In [ ]:
#giving engine answer and role template
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": new_summary_tmpl}
)

In [ ]:
#printing
prompts_dict = query_engine.get_prompts()
print(prompts_dict)

In [ ]:
#uncomment when you have acces for qestions in txt
# question = open(pth+str(0)+'.txt').read()
# qs = question[:question.find('**')].replace('\n\n',' ').split('\n')
# qs

In [ ]:
#asking chat a question
query_engine.query("What is the difference between CBOW and skip-gram?")